<a href="https://colab.research.google.com/github/Hab-eeb/Flower_color_detection/blob/main/color_det_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [8]:
import os
import sys
import datetime
import glob as glob
import numpy as np
import cv2
from collections import Counter

# import the Keras implementations of VGG16, VGG19, InceptionV3 and Xception models
import tensorflow 
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications import InceptionV3, ResNet50, Xception
from tensorflow.keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
from keras import layers, models, optimizers
import tensorflow.keras.backend as K
import itertools
from itertools import tee
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

#from scipy.interpolate import spline
import pandas as pd
import matplotlib.pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.utils import class_weight

# Importing Dataset from google drive

In [11]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! unzip "/content/drive/MyDrive/Flower_Color_Detection/Orchid Flowers Dataset-v1.1.zip"

# Model Building

In [14]:
# Definition for calculating class weights
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights

In [15]:
# weighted binary cross entropy definition
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

In [ ]:
# [Dataset]
# image dimensions for VGG16, VGG19 are 224, 224
# image dimensions for InceptionV3 and Xception are 299, 299
img_width, img_height = 224,224

train_dir = 'Training'
validate_dir = 'Validation'
nb_epochs = 100
batch_size = 64
nb_classes = len(glob.glob(train_dir + '/*'))

# get number of images in training directory
nb_train_samples = 0
for r, dirs, files in os.walk(train_dir):
    for dr in dirs:
        nb_train_samples += len(glob.glob(os.path.join(r, dr + "/*")))

# get number of images in validation directory
nb_validate_samples = 0
for r, dirs, files in os.walk(validate_dir):
    for dr in dirs:
        nb_validate_samples += len(glob.glob(os.path.join(r, dr + "/*")))

print(nb_train_samples)
print(nb_validate_samples)
print(nb_classes)

In [ ]:
# data pre-processing for training
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

# data pre-processing for validation
validate_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

testdatagen=ImageDataGenerator(rescale=1./255)

train_gener =train_datagen.flow_from_directory(
train_dir,
target_size=(img_height,img_width),
#the total amount should be divisable by batch size
batch_size=5119,
shuffle=False,
class_mode= "categorical"

In [ ]:
# generate and store training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       

(train_pics,train_labels) = next(train_gener)
class_weights = calculating_class_weights(train_labels) 

# generate and store validation data
validate_generator = validate_datagen.flow_from_directory(
    validate_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

In [ ]:
#%% define the model (see other evaluation_load file for the other models, and you can copy paste them here to use them)
xceptionmodel = Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(img_width, img_height,3)
)
for layer in xceptionmodel.layers[:2]:
    layer.trainable=False
    
xceptionmodel.summary()

x= xceptionmodel.output
x = Flatten(name='flatten')(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
predictions= Dense(nb_classes,activation='softmax')(x)

model= Model(inputs=[xceptionmodel.input], outputs= predictions)

In [ ]:
## Training

checkpoint = ModelCheckpoint( "weights3.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
model.compile(loss=get_weighted_loss(class_weights), optimizer=optimizers.Adam(learning_rate = 0.00005), metrics = ['accuracy'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validate_generator.n//validate_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

metrics = Metrics(validate_generator, STEP_SIZE_VALID)

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validate_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=nb_epochs,
                # callbacks=[metrics, checkpoint]
)